In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from fastai.tabular.core import add_datepart
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Import data

In [7]:
data = pd.read_json('data/data.json')

### Convert 'acct_type' to Target Labels based on presence of word "fraud"

In [8]:
data['Label'] = np.where(data['acct_type'].str.contains('fraud'), 1, 0)

In [9]:
data.head(1)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,Label
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums/qq49/digusonline/?action=view&amp;current=supersunday.jpg"" target=""_blank""><img src=""http://i432.photobucket.com/albums/qq49/digusonline/supersunday.jpg"" border=""0"" alt=""Photobucket"" /></a></p>\r\n<p> </p>\r\n<p style=""text-align: center;""><font size=""3""><strong>Party Starz Entertaintment &amp; Diverse International Group Presents...<br /></strong></font><br /><font face=""tahoma,arial,helvetica,sans-serif"" size=""4""><strong>The Official ""99 Hour No Sleep"" Super Bowl </strong></font></p>\r\n<p style=""text-align: center;""><font face=""tahoma,arial...",gmail.com,1262739706,...,36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,1


### Define replace function for handling missing (non-NaN) data

In [10]:
def replace(row):
    if len(row) < 1:
        return "UNK"
    else: return row

In [11]:
data['payout_type'] = data['payout_type'].apply(replace)
data['country'].fillna('UNK', inplace=True)
data['country'] = data['country'].apply(replace)
data['venue_state'].fillna('UNK', inplace=True)
data['venue_state'] = data['venue_state'].apply(replace)

### Time Series conversions

In [12]:
data['user_created'] = pd.to_datetime(data['user_created'], unit='s')
data['approx_payout_date'] = pd.to_datetime(data['approx_payout_date'], unit = 's')
data['event_created'] = pd.to_datetime(data['event_created'], unit='s')
data['event_end']= pd.to_datetime(data['event_end'], unit='s')
data['event_published'] = pd.to_datetime(data['event_published'], unit='s')
data['event_start'] = pd.to_datetime(data['event_start'], unit='s')
data['time_diff'] = data['event_end'] - data['event_start']
data['time_diff_days'] = data['time_diff'].dt.days
data['time_diff_seconds'] = data['time_diff'].dt.total_seconds()

In [13]:
data['year'] = data['approx_payout_date'].dt.year
data['month'] = data['approx_payout_date'].dt.month
data['day'] = data['approx_payout_date'].dt.day
data['hour'] = data['approx_payout_date'].dt.hour
data['minute'] = data['approx_payout_date'].dt.minute
data['dayofyear'] = data['approx_payout_date'].dt.dayofyear
data['weekofyear'] = data['approx_payout_date'].dt.weekofyear
data['week'] = data['approx_payout_date'].dt.week
data['dayofweek'] = data['approx_payout_date'].dt.dayofweek
data['quarter'] = data['approx_payout_date'].dt.quarter
data['is_month_start'] = data['approx_payout_date'].dt.is_month_start
data['is_month_end'] = data['approx_payout_date'].dt.is_month_end
data['is_quarter_start'] = data['approx_payout_date'].dt.is_quarter_start
data['is_quarter_end'] = data['approx_payout_date'].dt.is_quarter_end

<ipython-input-13-5489717b1398>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['weekofyear'] = data['approx_payout_date'].dt.weekofyear
<ipython-input-13-5489717b1398>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['week'] = data['approx_payout_date'].dt.week


### Separate datasets into fraud and non-fraud for some visual EDA

In [14]:
fraud = data[data['Label'] == 1]
nonfraud = data[data['Label'] == 0]

### Define columns for histogram plots

In [15]:
cols = [
    'acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_facebook', 'org_twitter',
       'payee_name', 'payout_type', 'sale_duration',
       'sale_duration2', 'show_map', 'user_age',
       'user_created', 'user_type',  'venue_country', 'venue_state'
]

In [16]:
def hist_plots(fraud_df, nonfraud_df, column, bins=30):
    '''
    Plots a comparison histogram on same plot between fraud and nonfraudulent events
    '''
    
    plt.hist(nonfraud_df[column], label="Class #0", alpha=0.5, edgecolor='black', lw=1.5, color='blue', bins=bins)
    plt.hist(fraud_df[column], label="Class #1", alpha=0.5, edgecolor='black', linewidth=1.5, color='r', bins=bins)
    plt.title(column, fontsize=34)
    plt.legend(fontsize='xx-large')
    return plt.show()

In [17]:
# for col in cols:
#     try:
#         hist_plots(fraud, nonfraud, col)
#     except TypeError:
#         pass

#### **** Look at org name, previous payouts, any text based columns, ticket types, lat, long, venue address and name, separately ****

### Convert ticket_type column into Dataframe for analysis

In [18]:
pd.DataFrame(fraud.ticket_types[0])

,event_id,cost,availability,quantity_total,quantity_sold
0,527017,25.0,1,800,0
1,527017,50.0,1,100,0
2,527017,550.0,1,20,0


### Fraud conversion + analytsis

In [19]:
frame_list = []
for x in fraud.ticket_types.values:
    frame_list.append(pd.DataFrame(x))

fraud_ticket_types = pd.concat(frame_list)

frame_list = []
for x in nonfraud.ticket_types.values:
    frame_list.append(pd.DataFrame(x))

nonfraud_ticket_types = pd.concat(frame_list)

fraud_ticket_types.reset_index(drop=True, inplace=True)
nonfraud_ticket_types.reset_index(drop=True, inplace=True)

fraud_ticket_types.iloc[:, 1:].describe()
nonfraud_ticket_types.iloc[:, 1:].describe()

,cost,availability,quantity_total,quantity_sold
count,36392.000000,36392.000000,36392.000000,36392.000000
mean,138.091798,1.050561,730.463591,26.456309
std,968.904638,0.219102,11188.863610,135.592463
min,-1.160000,1.000000,0.000000,0.000000
25%,10.000000,1.000000,20.000000,1.000000
50%,22.850000,1.000000,50.000000,4.000000
75%,69.720000,1.000000,121.000000,17.000000
max,100000.000000,2.000000,500000.000000,10606.000000


In [20]:
fraud_ticket_types['Label'] = 1
nonfraud_ticket_types['Label'] = 0

In [21]:
ticket_types = pd.concat([fraud_ticket_types, nonfraud_ticket_types]).reset_index(drop=True)

In [22]:
ticket_types.drop('event_id', axis=1, inplace=True)

In [23]:
ticket_types[ticket_types['Label'] == 1]['availability'].value_counts()

1    2225
2      12
Name: availability, dtype: int64

In [24]:
ticket_types[ticket_types['Label'] == 0]['availability'].value_counts()

1    34552
2     1840
Name: availability, dtype: int64

In [25]:
ticketx = ticket_types.iloc[:, :-1].values
tickety = ticket_types['Label'].values

In [26]:
trainx, testx, trainy, testy = train_test_split(ticketx, tickety, test_size=0.2, stratify=tickety)
trainx.shape, testx.shape, trainy.shape, testy.shape

((30903, 4), (7726, 4), (30903,), (7726,))

In [27]:
lr = LogisticRegression(n_jobs=-1, verbose=1)
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, verbose=1)

In [28]:
print(cross_val_score(lr, trainx, trainy, cv=10, scoring='f1', verbose=1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend 

[0.10416667 0.13541667 0.12565445 0.11578947 0.125      0.10471204
 0.0952381  0.04347826 0.16326531 0.16080402]


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.9s finished


In [29]:
print(cross_val_score(rf, trainx, trainy, cv=5, scoring='f1', verbose=1, n_jobs=1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 

[0.40070922 0.38408304 0.42692939 0.35978836 0.40468227]


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.3s finished


### Time Series Analysis

In [30]:
time_cols = ['year', 'month', 'day', 'hour', 'minute', 'dayofyear',
       'weekofyear', 'week', 'dayofweek', 'quarter', 'is_month_start',
       'is_month_end', 'is_quarter_start', 'is_quarter_end']

In [31]:
# for col in time_cols:
#     try:
#         hist_plots(fraud, nonfraud, col)
#     except TypeError:
#         pass

In [32]:
data.head(1)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,minute,dayofyear,weekofyear,week,dayofweek,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end
0,fraudster_event,2010-02-13 12:00:00,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums/qq49/digusonline/?action=view&amp;current=supersunday.jpg"" target=""_blank""><img src=""http://i432.photobucket.com/albums/qq49/digusonline/supersunday.jpg"" border=""0"" alt=""Photobucket"" /></a></p>\r\n<p> </p>\r\n<p style=""text-align: center;""><font size=""3""><strong>Party Starz Entertaintment &amp; Diverse International Group Presents...<br /></strong></font><br /><font face=""tahoma,arial,helvetica,sans-serif"" size=""4""><strong>The Official ""99 Hour No Sleep"" Super Bowl </strong></font></p>\r\n<p style=""text-align: center;""><font face=""tahoma,arial...",gmail.com,2010-01-06 01:01:46,...,0,44,6,6,5,1,False,False,False,False


In [33]:
model_cols.previous_payouts

NameError: name 'model_cols' is not defined

In [34]:
data.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state',
       'Label', 'time_diff', 'time_diff_days', 'time_diff_seconds', 'year',
       'month', 'day', 'hour', 'minute', 'dayofyear', 'weekofyear', 'week',
       'dayofweek', 'quarter', 'is_month_start', 'is_month_end',
       'is_quarter_start', 'is_quarter_end'],
      dtype='object'

In [35]:
features = ['body_length', 'channels', 'country',
       'currency', 'delivery_method', 'email_domain',
       'fb_published', 'has_header', 'has_logo', 'name_length',
       'object_id', 'org_facebook', 'org_twitter',
       'show_map', 'user_age',
       'user_created', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_state',
       'Label', 'year', 'month', 'day', 'hour', 'minute', 'dayofyear',
       'weekofyear', 'week', 'dayofweek', 'quarter', 'is_month_start',
       'is_month_end', 'is_quarter_start', 'is_quarter_end', 'time_diff',
       'time_diff_seconds', 'previous_payout0']

In [36]:
def compare(frauddf, nonfrauddf, col):
    print(f'Fraud: \n{frauddf[col].value_counts()/len(frauddf)*100} \n\n\nNonFraud: \n{nonfrauddf[col].value_counts()/len(nonfrauddf) * 100}\n\n')

In [37]:
compare(fraud, nonfraud, 'user_type')

Fraud: 
1    71.771075
3    22.815159
4     5.104408
5     0.232019
2     0.077340
Name: user_type, dtype: float64 


NonFraud: 
3      55.389451
4      22.623428
1      21.542472
5       0.429316
103     0.015333
Name: user_type, dtype: float64




In [38]:
len(nonfraud[nonfraud['num_order'] == 0])/len(nonfraud), len(nonfraud[nonfraud['num_order'] == 0]), len(nonfraud)

(0.002836553204538485, 37, 13044)

In [39]:
len(fraud[fraud['num_order'] == 0])/len(fraud), len(fraud[fraud['num_order'] == 0]), len(fraud)

(0.24748646558391338, 320, 1293)

### Text columns:
- description
- name
- org_description
- org_name
- venue address
- venue_name

### Leaky columns
- num_order
- num_payouts
- gts

In [40]:
columns_removed = set(data.columns).difference(features)
columns_removed

{'acct_type',
 'approx_payout_date',
 'description',
 'event_created',
 'event_end',
 'event_published',
 'event_start',
 'gts',
 'has_analytics',
 'listed',
 'name',
 'num_order',
 'num_payouts',
 'org_desc',
 'org_name',
 'payee_name',
 'payout_type',
 'previous_payouts',
 'sale_duration',
 'sale_duration2',
 'ticket_types',
 'time_diff_days',
 'user_type',
 'venue_address',
 'venue_name'}

### Feature engineering List:
- num order
- gts

In [41]:
model_cols.drop(['org_name', 'ticket_types', 'venue_address', 'venue_name'], axis=1, inplace=True)

NameError: name 'model_cols' is not defined

In [42]:
model_cols['previous_payout0'] = np.where(truth_col, 1, 0)

NameError: name 'truth_col' is not defined

In [ ]:
event_cols = model_cols.columns[model_cols.columns.str.contains('event')]

In [43]:
model_cols.drop(event_cols, axis=1, inplace=True)

NameError: name 'model_cols' is not defined

In [ ]:
model_cols.has_header.fillna('UNK', inplace=True)

In [44]:
model_cols.has_header.value_counts()

NameError: name 'model_cols' is not defined

In [ ]:
fraud = model_cols[model_cols['Label'] == 1]
nonfraud = model_cols[model_cols['Label'] == 0]

In [45]:
fraud_header = compare(fraud, nonfraud, 'has_header')

Fraud: 
0.0    37.354988
1.0     2.784223
Name: has_header, dtype: float64 


NonFraud: 
0.0    50.912297
1.0    13.554125
Name: has_header, dtype: float64




In [46]:
dummy_cols = pd.get_dummies(model_cols, columns=['has_header'], dummy_na=True)

NameError: name 'model_cols' is not defined

In [47]:
dummy_cols.drop(['country', 'currency', 'previous_payouts'], axis=1, inplace=True)

NameError: name 'dummy_cols' is not defined

In [247]:
dummy_cols.drop('time_diff', axis=1, inplace=True)

In [249]:
dummy_cols.head()

,body_length,channels,delivery_method,email_domain,fb_published,has_logo,name_length,object_id,org_facebook,org_twitter,show_map,user_age,user_created,user_type,venue_country,venue_latitude,venue_longitude,venue_state,Label,year,month,day,hour,minute,dayofyear,weekofyear,week,dayofweek,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,time_diff_seconds,previous_payout0,has_header_0.0,has_header_1.0,has_header_nan
0,3852,5,0.0,gmail.com,0,0,60,527017,0.0,0.0,1,36,1259613950,1,US,25.777471,-80.133433,FL,1,2010,2,13,12,0,44,6,6,5,1,False,False,False,False,36000.0,1,0,1,0
1,3499,0,1.0,ruf.org,0,1,27,786878,0.0,12.0,0,149,1280942776,3,US,32.776566,-79.930922,SC,0,2011,2,3,8,0,34,5,5,3,1,False,False,False,False,32400.0,0,1,0,0
2,2601,8,1.0,pvsd.k12.ca.us,0,0,28,787337,0.0,0.0,0,214,1272559388,3,US,33.944201,-118.080419,CA,0,2011,1,28,0,0,28,4,4,4,1,False,False,False,False,27000.0,0,0,0,1
3,12347,6,1.0,irishtabletennis.com,0,1,21,885645,0.0,0.0,0,889,1283870102,3,None,NaN,NaN,UNK,0,2014,1,6,0,0,6,2,2,0,1,False,False,False,False,27831600.0,0,1,0,0
4,2417,11,0.0,artsandbusinesscouncil.org,1,0,66,1114349,0.0,0.0,0,35,1288984065,3,US,42.353848,-71.044276,MA,0,2011,2,17,0,0,48,7,7,3,1,False,False,False,False,28800.0,0,1,0,0


In [256]:
dummy_cols['yahoo'] = np.where(model_cols['email_domain'].str.contains('yahoo'), 1, 0)

In [272]:
dummy_cols['is_fr'] = np.where(model_cols['email_domain'].str.endswith('fr'), 1, 0)

In [275]:
dummy_cols['is_org'] = np.where(model_cols['email_domain'].str.endswith('.org'), 1, 0)

In [281]:
data['user_created'].dt.year

0        2009
1        2010
2        2010
3        2010
4        2010
         ... 
14332    2013
14333    2009
14334    2012
14335    2010
14336    2013
Name: user_created, Length: 14337, dtype: int64

In [290]:
dummy_cols.drop(['email_domain', 'object_id', 'user_created', 'venue_country', 'venue_latitude', 'venue_longitude', 'venue_state'], axis=1, inplace=True)

In [304]:
dummy_cols.drop(['year'], axis=1, inplace=True)

In [332]:
# filled_nas = dummy_cols
# filled_nas.org_facebook.fillna(-1, inplace=True)
# filled_nas.org_twitter.fillna(-1, inplace=True)
filled_nas.delivery_method.value_counts()

0.0    8520
1.0    5554
3.0     247
Name: delivery_method, dtype: int64

In [335]:
filled_nas = pd.get_dummies(filled_nas, columns=['delivery_method'], dummy_na=True)

In [356]:
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, verbose=1)

In [1]:
filled_nas

NameError: name 'filled_nas' is not defined

In [353]:
X = filled_nas.drop('Label', axis=1).values
y = filled_nas['Label'].values

In [354]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [357]:
np.mean(cross_val_score(rf, xtrain, ytrain, scoring='f1', cv=5, verbose=1, n_jobs=4))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    2.2s finished


0.8938278572422658

In [342]:
filled_nas.columns

Index(['body_length', 'channels', 'fb_published', 'has_logo', 'name_length',
       'org_facebook', 'org_twitter', 'show_map', 'user_age', 'user_type',
       'Label', 'month', 'day', 'hour', 'minute', 'dayofyear', 'weekofyear',
       'week', 'dayofweek', 'quarter', 'is_month_start', 'is_month_end',
       'is_quarter_start', 'is_quarter_end', 'time_diff_seconds',
       'previous_payout0', 'has_header_0.0', 'has_header_1.0',
       'has_header_nan', 'yahoo', 'is_fr', 'is_org', 'delivery_method_0.0',
       'delivery_method_1.0', 'delivery_method_3.0', 'delivery_method_nan'],
      dtype='object')

In [359]:
filled_nas.to_csv('data/final_columns.csv', index=False)

In [358]:
ls

EDA_Sanchez.ipynb             example.json
FraudCaseStudyOverview.ipynb  example_app.py
Pickle.ipynb                  for_instructors/
Sams-notebook.ipynb           model_notes.md
Sanchez_notebook.ipynb        pair.md
Stephen_notebook.ipynb        readme.md
__pycache__/                  stephen_eda.py
assignment.md                 subset_json.py
data/
